In [ ]:
# !pip install ../input/birds-inference-pip-wheels/torchaudio-0.8.1-cp37-cp37m-manylinux1_x86_64.whl ../input/birds-inference-pip-wheels/torch-1.8.1-cp37-cp37m-manylinux1_x86_64.whl
# !pip install ../input/birds-inference-pip-wheels/timm-0.4.8.zip --no-index --no-deps
# #!pip install ../input/birdclef21trainmeta/timm-0.4.9_23052021/pytorch-image-models-master --no-index --no-deps
# !pip install ../input/birds-inference-pip-wheels/audiomentations-0.16.0-py3-none-any.whl --no-index --no-deps
# !pip install ../input/birds-inference-pip-wheels/torchlibrosa-0.0.9-py3-none-any.whl --no-index --no-deps

In [ ]:
import timm
timm.__version__

'0.6.13'

# Imports

In [ ]:
import pandas as pd
import timm
from torch import nn
import torch
import torchaudio as ta
from torch.cuda.amp import autocast
import random

from torch.nn import functional as F
from torch.distributions import Beta
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset

import numpy as np
import librosa
import ast

import os
from types import SimpleNamespace
import numpy as np

import numpy as np
import pandas as pd
import importlib
import sys
import random
from tqdm import tqdm
import gc
import argparse
import torch
from torch import optim
from torch.cuda.amp import GradScaler, autocast
from collections import defaultdict
import cv2
from copy import copy
import os
from transformers import get_cosine_schedule_with_warmup
from torch.utils.data import SequentialSampler, DataLoader
import warnings
import torchaudio
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
def set_seed(seed=1234):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

# Config

In [ ]:
cfg = SimpleNamespace()

# paths
cfg.data_folder = ''
cfg.name = "julian"
cfg.data_dir = "../input/birdclef-2023/"
cfg.train_data_folder = cfg.data_dir + "train_audio/"
cfg.val_data_folder = cfg.data_dir + "train_audio/"
cfg.output_dir = "pre_model_multi_drop_v4"

# dataset
cfg.dataset = "base_ds"
cfg.min_rating = 0
cfg.val_df = None
cfg.batch_size_val = 1
cfg.train_aug = None
cfg.val_aug = None
cfg.test_augs = None
cfg.wav_len_val = 5  # seconds
cfg.min_rating = 2.0
cfg.wav_crop_len = 30  # seconds
# audio
cfg.img_height = 256
cfg.img_weight = 512
cfg.window_size = 2048
cfg.sample_rate = 32000
cfg.fmin = 16
cfg.fmax = 16386
cfg.power = 2
cfg.mel_bins = cfg.img_height
cfg.hop_size = 512 # int(cfg.sample_rate * cfg.wav_crop_len / (cfg.img_weight - 1))
cfg.top_db = 80.0

# img model

cfg.pretrained = True
cfg.pretrained_weights = None
cfg.train = True
cfg.val = False
cfg.in_chans = 1

cfg.alpha = 1
cfg.eval_epochs = 1
cfg.eval_train_epochs = 1
cfg.warmup = 0

cfg.mel_norm = False

cfg.label_smoothing = 0

cfg.remove_pretrained = []

# training
cfg.seed = 123
cfg.save_val_data = True

# ressources
cfg.mixed_precision = True
cfg.gpu = 0
cfg.num_workers = 4 # 18
cfg.drop_last = True

cfg.mixup2 = 0

cfg.label_smoothing = 0

cfg.mixup_2x = False


cfg.birds = np.array(['abethr1', 'abhori1', 'abythr1', 'afbfly1', 'afdfly1', 'afecuc1',
       'affeag1', 'afgfly1', 'afghor1', 'afmdov1', 'afpfly1', 'afpkin1',
       'afpwag1', 'afrgos1', 'afrgrp1', 'afrjac1', 'afrthr1', 'amesun2',
       'augbuz1', 'bagwea1', 'barswa', 'bawhor2', 'bawman1', 'bcbeat1',
       'beasun2', 'bkctch1', 'bkfruw1', 'blacra1', 'blacuc1', 'blakit1',
       'blaplo1', 'blbpuf2', 'blcapa2', 'blfbus1', 'blhgon1', 'blhher1',
       'blksaw1', 'blnmou1', 'blnwea1', 'bltapa1', 'bltbar1', 'bltori1',
       'blwlap1', 'brcale1', 'brcsta1', 'brctch1', 'brcwea1', 'brican1',
       'brobab1', 'broman1', 'brosun1', 'brrwhe3', 'brtcha1', 'brubru1',
       'brwwar1', 'bswdov1', 'btweye2', 'bubwar2', 'butapa1', 'cabgre1',
       'carcha1', 'carwoo1', 'categr', 'ccbeat1', 'chespa1', 'chewea1',
       'chibat1', 'chtapa3', 'chucis1', 'cibwar1', 'cohmar1', 'colsun2',
       'combul2', 'combuz1', 'comsan', 'crefra2', 'crheag1', 'crohor1',
       'darbar1', 'darter3', 'didcuc1', 'dotbar1', 'dutdov1', 'easmog1',
       'eaywag1', 'edcsun3', 'egygoo', 'equaka1', 'eswdov1', 'eubeat1',
       'fatrav1', 'fatwid1', 'fislov1', 'fotdro5', 'gabgos2', 'gargan',
       'gbesta1', 'gnbcam2', 'gnhsun1', 'gobbun1', 'gobsta5', 'gobwea1',
       'golher1', 'grbcam1', 'grccra1', 'grecor', 'greegr', 'grewoo2',
       'grwpyt1', 'gryapa1', 'grywrw1', 'gybfis1', 'gycwar3', 'gyhbus1',
       'gyhkin1', 'gyhneg1', 'gyhspa1', 'gytbar1', 'hadibi1', 'hamerk1',
       'hartur1', 'helgui', 'hipbab1', 'hoopoe', 'huncis1', 'hunsun2',
       'joygre1', 'kerspa2', 'klacuc1', 'kvbsun1', 'laudov1', 'lawgol',
       'lesmaw1', 'lessts1', 'libeat1', 'litegr', 'litswi1', 'litwea1',
       'loceag1', 'lotcor1', 'lotlap1', 'luebus1', 'mabeat1', 'macshr1',
       'malkin1', 'marsto1', 'marsun2', 'mcptit1', 'meypar1', 'moccha1',
       'mouwag1', 'ndcsun2', 'nobfly1', 'norbro1', 'norcro1', 'norfis1',
       'norpuf1', 'nubwoo1', 'pabspa1', 'palfly2', 'palpri1', 'piecro1',
       'piekin1', 'pitwhy', 'purgre2', 'pygbat1', 'quailf1', 'ratcis1',
       'raybar1', 'rbsrob1', 'rebfir2', 'rebhor1', 'reboxp1', 'reccor',
       'reccuc1', 'reedov1', 'refbar2', 'refcro1', 'reftin1', 'refwar2',
       'rehblu1', 'rehwea1', 'reisee2', 'rerswa1', 'rewsta1', 'rindov',
       'rocmar2', 'rostur1', 'ruegls1', 'rufcha2', 'sacibi2', 'sccsun2',
       'scrcha1', 'scthon1', 'shesta1', 'sichor1', 'sincis1', 'slbgre1',
       'slcbou1', 'sltnig1', 'sobfly1', 'somgre1', 'somtit4', 'soucit1',
       'soufis1', 'spemou2', 'spepig1', 'spewea1', 'spfbar1', 'spfwea1',
       'spmthr1', 'spwlap1', 'squher1', 'strher', 'strsee1', 'stusta1',
       'subbus1', 'supsta1', 'tacsun1', 'tafpri1', 'tamdov1', 'thrnig1',
       'trobou1', 'varsun2', 'vibsta2', 'vilwea1', 'vimwea1', 'walsta1',
       'wbgbir1', 'wbrcha2', 'wbswea1', 'wfbeat1', 'whbcan1', 'whbcou1',
       'whbcro2', 'whbtit5', 'whbwea1', 'whbwhe3', 'whcpri2', 'whctur2',
       'wheslf1', 'whhsaw1', 'whihel1', 'whrshr1', 'witswa1', 'wlwwar',
       'wookin1', 'woosan', 'wtbeat1', 'yebapa1', 'yebbar1', 'yebduc1',
       'yebere1', 'yebgre1', 'yebsto1', 'yeccan1', 'yefcan', 'yelbis1',
       'yenspu1', 'yertin1', 'yesbar1', 'yespet1', 'yetgre1', 'yewgre1'])


cfg.n_classes = len(cfg.birds)


# training
cfg.lr = 0.001
cfg.epochs = 30
cfg.batch_size = 64
cfg.batch_size_val = 16
cfg.backbone = "resnet34"


cfg.save_val_data = True
cfg.mixed_precision = True

cfg.mixup = True
cfg.mix_beta = 1


cfg.train_df1 = "../input/birdclef-2023/train_metadata.csv"
cfg.train_df2 = "../input/birdclef-22-duration/train_metadata_with_duration.csv"


cfg.device = 'cuda' if torch.cuda.is_available() else 'cpu'

cfg.tr_collate_fn = None
cfg.val_collate_fn = None
cfg.val = False

cfg.dev = False

cfg

namespace(data_folder='',
          name='julian',
          data_dir='../input/birdclef-2023/',
          train_data_folder='../input/birdclef-2023/train_audio/',
          val_data_folder='../input/birdclef-2023/train_audio/',
          output_dir='pre_model_multi_drop_v4',
          dataset='base_ds',
          min_rating=2.0,
          val_df=None,
          batch_size_val=16,
          train_aug=None,
          val_aug=None,
          test_augs=None,
          wav_len_val=5,
          wav_crop_len=30,
          img_height=256,
          img_weight=512,
          window_size=2048,
          sample_rate=32000,
          fmin=16,
          fmax=16386,
          power=2,
          mel_bins=256,
          hop_size=512,
          top_db=80.0,
          pretrained=True,
          pretrained_weights=None,
          train=True,
          val=False,
          in_chans=1,
          alpha=1,
          eval_epochs=1,
          eval_train_epochs=1,
          warmup=0,
          mel_norm=False,


# Dataset

In [ ]:
def batch_to_device(batch, device):
    batch_dict = {key: batch[key].to(device) for key in batch}
    return batch_dict



class CustomDataset(Dataset):
    def __init__(self, df, cfg, aug, mode="train"):

        self.cfg = cfg
        self.mode = mode
        self.df = df.copy()

        self.bird2id = {bird: idx for idx, bird in enumerate(cfg.birds)}
        if self.mode == "train":
            self.data_folder = cfg.train_data_folder
            self.df = self.df[self.df["rating"] >= self.cfg.min_rating]
        elif self.mode == "val":
            self.data_folder = cfg.val_data_folder
        elif self.mode == "test":
            self.data_folder = cfg.test_data_folder

        self.fns = self.df["filename"].unique()

        self.df = self.setup_df()

        self.aug_audio = cfg.train_aug

    def setup_df(self):
        df = self.df.copy()

        if self.mode == "train":

            df["weight"] = np.clip(df["rating"] / df["rating"].max(), 0.1, 1.0)
            df['target'] = df['primary_label'].apply(self.bird2id.get)
            labels = np.eye(self.cfg.n_classes)[df["target"].astype(int).values]
            label2 = df["secondary_labels"].apply(lambda x: self.secondary2target(x)).values
            for i, t in enumerate(label2):
                labels[i, t] = 1
        else:
            targets = df["birds"].apply(lambda x: self.birds2target(x)).values
            labels = np.zeros((df.shape[0], self.cfg.n_classes))
            for i, t in enumerate(targets):
                labels[i, t] = 1

        df[[f"t{i}" for i in range(self.cfg.n_classes)]] = labels

        if self.mode != "train":
            df = df.groupby("filename")

        return df

    def __getitem__(self, idx):

        if self.mode == "train":
            row = self.df.iloc[idx]
            fn = row["filename"]
            label = row[[f"t{i}" for i in range(self.cfg.n_classes)]].values
            weight = row["weight"]
            #fold = row["fold"]
            fold = -1

            #wav_len = row["length"]
            parts = 1
        else:
            fn = self.fns[idx]
            row = self.df.get_group(fn)
            label = row[[f"t{i}" for i in range(self.cfg.n_classes)]].values
            wav_len = None
            parts = label.shape[0]
            fold = -1
            weight = 1

        if self.mode == "train":
            #wav_len_sec = wav_len / self.cfg.sample_rate
            wav_len_sec = row['duration']
            duration = self.cfg.wav_crop_len
            max_offset = wav_len_sec - duration
            max_offset = max(max_offset, 1)
            offset = np.random.randint(max_offset)
        else:
            offset = 0.0
            duration = None

        wav = self.load_one(fn, offset, duration)

        if wav.shape[0] < (self.cfg.wav_crop_len * self.cfg.sample_rate):
            pad = self.cfg.wav_crop_len * self.cfg.sample_rate - wav.shape[0]
            wav = np.pad(wav, (0, pad))

        if self.mode == "train":
            if self.aug_audio:
                wav = self.aug_audio(samples=wav, sample_rate=self.cfg.sample_rate)
        else:
            if self.cfg.val_aug:
                wav = self.cfg.val_aug(samples=wav, sample_rate=self.cfg.sample_rate)

        wav_tensor = torch.tensor(wav)  # (n_samples)
        if parts > 1:
            n_samples = wav_tensor.shape[0]
            wav_tensor = wav_tensor[: n_samples // parts * parts].reshape(
                parts, n_samples // parts
            )

        feature_dict = {
            "input": wav_tensor,
            "target": torch.tensor(label.astype(np.float32)),
            "weight": torch.tensor(weight),
            "fold": torch.tensor(fold),
        }
        return feature_dict

    def __len__(self):
        if cfg.dev:
            return 256
        return len(self.fns)

    def load_one(self, id_, offset, duration):
        fp = self.data_folder + id_
        try:
            wav, sr = librosa.load(fp, sr=None, offset=offset, duration=duration)
        except:
            print("FAIL READING rec", fp)

        return wav

    def birds2target(self, birds):
        birds = birds.split()
        target = [self.bird2id.get(item) for item in birds if not item == "nocall"]
        return target

    def secondary2target(self, secondary_label):
        birds = ast.literal_eval(secondary_label)
        target = [self.bird2id.get(item) for item in birds if not item == "nocall"]
        return target


# Dataset related utils

In [ ]:
def worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)


def get_train_dataloader(train_ds, cfg):
    train_dataloader = DataLoader(
        train_ds,
        sampler=None,
        shuffle=True,
        batch_size=cfg.batch_size,
        num_workers=cfg.num_workers,
        pin_memory=False,
        collate_fn=cfg.tr_collate_fn,
        drop_last=cfg.drop_last,
        worker_init_fn=worker_init_fn,
    )
    print(f"train: dataset {len(train_ds)}, dataloader {len(train_dataloader)}")
    return train_dataloader


def get_scheduler(cfg, optimizer, total_steps):
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=cfg.warmup * (total_steps // cfg.batch_size),
        num_training_steps=cfg.epochs * (total_steps // cfg.batch_size),
    )
    return scheduler

def get_file_duration(file_path):
    sound, sr = librosa.load(file_path)

    return librosa.get_duration(y=sound, sr=sr)

def load_df(cfg):
    train_df1 = pd.read_csv(cfg.train_df1)
    #train_df1['duration'] = train_df1['filename'].apply(lambda x:get_file_duration(cfg.train_data_folder+x))
    train_df2 = pd.read_csv(cfg.train_df2, names=train_df1.columns.tolist() + ['duration'])
    train_df = pd.merge(train_df1[['primary_label', 'secondary_labels', 'rating', 'filename']], train_df2[['filename', 'duration']], how='inner', on='filename')
    return train_df


# GeM and Mix-up

In [ ]:
def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1.0 / p)


class GeM(nn.Module):
    # Generalized mean: https://arxiv.org/abs/1711.02512
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        ret = gem(x, p=self.p, eps=self.eps)
        return ret

    def __repr__(self):
        return (self.__class__.__name__+ "(p="+ "{:.4f}".format(self.p.data.tolist()[0])+ ", eps="+ str(self.eps)+ ")")


class Mixup(nn.Module):
    def __init__(self, mix_beta):

        super(Mixup, self).__init__()
        self.beta_distribution = Beta(mix_beta, mix_beta)

    def forward(self, X, Y, weight=None):

        bs = X.shape[0]
        n_dims = len(X.shape)
        perm = torch.randperm(bs)
        coeffs = self.beta_distribution.rsample(torch.Size((bs,))).to(X.device)

        if n_dims == 2:
            X = coeffs.view(-1, 1) * X + (1 - coeffs.view(-1, 1)) * X[perm]
        elif n_dims == 3:
            X = coeffs.view(-1, 1, 1) * X + (1 - coeffs.view(-1, 1, 1)) * X[perm]
        else:
            X = coeffs.view(-1, 1, 1, 1) * X + (1 - coeffs.view(-1, 1, 1, 1)) * X[perm]

        Y = coeffs.view(-1, 1) * Y + (1 - coeffs.view(-1, 1)) * Y[perm]

        if weight is None:
            return X, Y
        else:
            weight = coeffs.view(-1) * weight + (1 - coeffs.view(-1)) * weight[perm]
            return X, Y, weight


# Model

In [ ]:
class Net(nn.Module):
    def __init__(self, cfg):
        super(Net, self).__init__()

        self.cfg = cfg

        self.n_classes = cfg.n_classes

        self.mel_spec = ta.transforms.MelSpectrogram(
            sample_rate=cfg.sample_rate,
            n_fft=cfg.window_size,
            win_length=cfg.window_size,
            f_min=cfg.fmin,
            f_max=cfg.fmax,
            pad=0,
            n_mels=cfg.mel_bins,
            hop_length=cfg.hop_size,
            power=cfg.power,
            normalized=False,
        )

        self.amplitude_to_db = ta.transforms.AmplitudeToDB(top_db=cfg.top_db)
        self.wav2img = torch.nn.Sequential(self.mel_spec, self.amplitude_to_db)
        self.masking = torchaudio.transforms.FrequencyMasking(freq_mask_param=10)
        self.backbone = timm.create_model(
            cfg.backbone,
            pretrained=cfg.pretrained,
            num_classes=0,
            global_pool="",
            in_chans=cfg.in_chans,
        )

        if "efficientnet" in cfg.backbone:
            backbone_out = self.backbone.num_features
        else:
            backbone_out = self.backbone.feature_info[-1]["num_chs"]

        self.global_pool = GeM()

        self.dropout = nn.Dropout(0.2)
        self.dropout_1 = nn.Dropout(0.1)
        self.dropout_2 = nn.Dropout(0.2)
        self.dropout_3 = nn.Dropout(0.3)
        self.dropout_4 = nn.Dropout(0.4)
        self.dropout_5 = nn.Dropout(0.5)
        self.head_23 = nn.Linear(backbone_out, self.n_classes)

        if cfg.pretrained_weights is not None:
            sd = torch.load(cfg.pretrained_weights, map_location="cpu")["model"]
            sd = {k.replace("module.", ""): v for k, v in sd.items()}
            self.load_state_dict(sd, strict=True)
            print("weights loaded from", cfg.pretrained_weights)
        self.loss_fn = nn.BCEWithLogitsLoss(reduction="none")

        self.mixup = Mixup(mix_beta=cfg.mix_beta)

        self.factor = int(cfg.wav_crop_len / 5.0)

    def forward(self, batch):

        if not self.training:
            x = batch["input"]
            bs, parts, time = x.shape
            x = x.reshape(parts, time)
            y = batch["target"]
            y = y[0]
        else:

            x = batch["input"]
            y = batch["target"]
            bs, time = x.shape
            x = x.reshape(bs * self.factor, time // self.factor)

        with autocast(enabled=False):
            x = self.masking(self.wav2img(x))  # (bs, mel, time)
            if self.cfg.mel_norm:
                x = (x + 80) / 80

        x = x.permute(0, 2, 1)
        x = x[:, None, :, :]

        weight = batch["weight"]

        if self.training:
            b, c, t, f = x.shape
            x = x.permute(0, 2, 1, 3)
            x = x.reshape(b // self.factor, self.factor * t, c, f)

            if self.cfg.mixup:
                x, y, weight = self.mixup(x, y, weight)
            if self.cfg.mixup2:
                x, y, weight = self.mixup(x, y, weight)

            x = x.reshape(b, t, c, f)
            x = x.permute(0, 2, 1, 3)

        #print(x.shape)
        # multi drop out
        x = self.backbone(x)

#         x = (self.dropout_1(x) +
#              self.dropout_2(x) +
#              self.dropout_3(x) +
#              self.dropout_4(x) +
#              self.dropout_5(x)
#             ) / 5


        if self.training:
            b, c, t, f = x.shape
            x = x.permute(0, 2, 1, 3)
            x = x.reshape(b // self.factor, self.factor * t, c, f)
            x = x.permute(0, 2, 1, 3)
        x = self.global_pool(x)
        x = x[:, :, 0, 0]
        logits = self.head_23(self.dropout(x))

        loss = self.loss_fn(logits, y)
        loss = (loss.mean(dim=1) * weight) / weight.sum()
        loss = loss.sum()

        return {"loss": loss, "logits": logits.sigmoid(), "logits_raw": logits, "target": y}


In [ ]:
def create_checkpoint(model, optimizer, epoch, scheduler=None, scaler=None):
    checkpoint = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "epoch": epoch,
    }

    if scheduler is not None:
        checkpoint["scheduler"] = scheduler.state_dict()

    if scaler is not None:
        checkpoint["scaler"] = scaler.state_dict()
    return checkpoint

In [ ]:
class EMA():
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

In [ ]:

set_seed(cfg.seed)


#val_df = pd.read_csv(cfg.val_df)
#val_dataset = CustomDataset(val_df, cfg, aug=cfg.val_aug, mode="val")

train_df = load_df(cfg)

train_dataset = CustomDataset(train_df, cfg, aug=cfg.train_aug, mode="train")
train_dataloader = get_train_dataloader(train_dataset, cfg)
model = Net(cfg)
model.load_state_dict(torch.load('/kaggle/input/birdclef-22-duration/checkpoint_last_seed123_15.pth')['model'], strict=False)
model.to(cfg.device)

total_steps = len(train_dataset)

params = model.parameters()
## liner
def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=1e-5):

    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in model.backbone.named_parameters() if not any(nd in n for nd in no_decay)],
         'lr': encoder_lr, 'weight_decay': weight_decay},
        {'params': [p for n, p in model.backbone.named_parameters() if any(nd in n for nd in no_decay)],
         'lr': encoder_lr, 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if 'backbone' not in n],
         'lr': decoder_lr, 'weight_decay': weight_decay}
    ]
    return optimizer_parameters

optimizer_parameters = get_optimizer_params(model,
                                            encoder_lr=cfg.lr,
                                            decoder_lr=cfg.lr*1.2,
                                            weight_decay=0)
optimizer = optim.AdamW(optimizer_parameters, lr=cfg.lr, weight_decay=1e-5)
scheduler = get_scheduler(cfg, optimizer, total_steps)


device = cfg.device

train: dataset 16113, dataloader 251


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet34-43635321.pth" to /root/.cache/torch/hub/checkpoints/resnet34-43635321.pth


In [ ]:
#[i for i in model.named_parameters()]

In [ ]:
#!ls -l /root/.cache/torch/hub/checkpoints/

In [ ]:
#!cp /root/.cache/torch/hub/checkpoints/resnet34-43635321.pth .

In [ ]:
try:
    os.makedirs(cfg.output_dir)
except:
    pass

if cfg.mixed_precision:
    scaler = GradScaler()
else:
    scaler = None


cfg.curr_step = 0
i = 0
best_val_loss = np.inf
optimizer.zero_grad()
for epoch in range(cfg.epochs):

    set_seed(cfg.seed + epoch)

    cfg.curr_epoch = epoch

    print("EPOCH:", epoch)

    progress_bar = tqdm(range(len(train_dataloader)))
    tr_it = iter(train_dataloader)

    losses = []

    gc.collect()

    if cfg.train:
        # ==== TRAIN LOOP
        for itr in progress_bar:
            i += 1

            cfg.curr_step += cfg.batch_size

            data = next(tr_it)

            model.train()
            torch.set_grad_enabled(True)

            batch = batch_to_device(data, device)

            if cfg.mixed_precision:
                with autocast():
                    output_dict = model(batch)
            else:
                output_dict = model(batch)

            loss = output_dict["loss"]

            losses.append(loss.item())

            if cfg.mixed_precision:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            else:
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()

            if cfg.curr_step % cfg.batch_size == 0:
                progress_bar.set_description(f"loss: {np.mean(losses[-10:]):.4f}")

    if cfg.val:
        if (epoch + 1) % cfg.eval_epochs == 0 or (epoch + 1) == cfg.epochs:
            val_loss = run_eval(model, val_dataloader, cfg)
        else:
            val_score = 0

    if cfg.epochs > 0:
        checkpoint = create_checkpoint(
            model, optimizer, epoch, scheduler=scheduler, scaler=scaler
        )

        torch.save(checkpoint, f"{cfg.output_dir}/checkpoint_last_seed{cfg.seed}_{epoch}.pth")

if cfg.epochs > 0:
    checkpoint = create_checkpoint(model, optimizer, epoch, scheduler=scheduler, scaler=scaler)

    torch.save(checkpoint, f"{cfg.output_dir}/checkpoint_last_seed{cfg.seed}.pth")

EPOCH: 0


loss: 0.0233: 100%|██████████| 251/251 [08:20<00:00,  1.99s/it]


EPOCH: 1


loss: 0.0218: 100%|██████████| 251/251 [07:50<00:00,  1.87s/it]


EPOCH: 2


...

loss: 0.0104: 100%|██████████| 251/251 [07:58<00:00,  1.91s/it]


EPOCH: 28


loss: 0.0104: 100%|██████████| 251/251 [07:58<00:00,  1.91s/it]


EPOCH: 29


loss: 0.0104: 100%|██████████| 251/251 [07:58<00:00,  1.91s/it]


## EMA

In [ ]:
# ema = EMA(model, 0.999)
# ema.register()

# try:
#     os.makedirs(cfg.output_dir)
# except:
#     pass

# if cfg.mixed_precision:
#     scaler = GradScaler()
# else:
#     scaler = None


# cfg.curr_step = 0
# i = 0
# best_val_loss = np.inf
# optimizer.zero_grad()
# for epoch in range(cfg.epochs):

#     set_seed(cfg.seed + epoch)

#     cfg.curr_epoch = epoch

#     print("EPOCH:", epoch)

#     progress_bar = tqdm(range(len(train_dataloader)))
#     tr_it = iter(train_dataloader)

#     losses = []

#     gc.collect()

#     if cfg.train:
#         # ==== TRAIN LOOP
#         for itr in progress_bar:
#             i += 1

#             cfg.curr_step += cfg.batch_size

#             data = next(tr_it)

#             model.train()
#             torch.set_grad_enabled(True)

#             batch = batch_to_device(data, device)

#             if cfg.mixed_precision:
#                 with autocast():
#                     output_dict = model(batch)
#             else:
#                 output_dict = model(batch)

#             loss = output_dict["loss"]

#             losses.append(loss.item())

#             if cfg.mixed_precision:
#                 scaler.scale(loss).backward()
#                 scaler.step(optimizer)
#                 scaler.update()
#                 optimizer.zero_grad()
#             else:
#                 loss.backward()
#                 optimizer.step()
#                 optimizer.zero_grad()

#             if scheduler is not None:
#                 scheduler.step()
#             ema.update()

#             if cfg.curr_step % cfg.batch_size == 0:
#                 progress_bar.set_description(f"loss: {np.mean(losses[-10:]):.4f}")

#     if cfg.val:
#         if (epoch + 1) % cfg.eval_epochs == 0 or (epoch + 1) == cfg.epochs:
#             val_loss = run_eval(model, val_dataloader, cfg)
#         else:
#             val_score = 0

#     if cfg.epochs > 0:
#         checkpoint = create_checkpoint(
#             model, optimizer, epoch, scheduler=scheduler, scaler=scaler
#         )

#         torch.save(checkpoint, f"{cfg.output_dir}/checkpoint_last_seed{cfg.seed}_{epoch}.pth")

# if cfg.epochs > 0:
#     checkpoint = create_checkpoint(model, optimizer, epoch, scheduler=scheduler, scaler=scaler)

#     torch.save(checkpoint, f"{cfg.output_dir}/checkpoint_last_seed{cfg.seed}.pth")